# Non-Negative Matric Factorization

### Data used- articles scraped from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('npr.csv')

In [3]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


### Preprocessing

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0

**`min_df`**` : float in range [0.0, 1.0] or int, default=1  

In [5]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [6]:
dtm = tfidf.fit_transform(npr['Article'])

In [7]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

### NMF

In [8]:
from sklearn.decomposition import NMF

In [9]:
nmf_model = NMF(n_components=9)

In [10]:
nmf_model.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=9, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

### Displaying Topics

In [11]:
len(tfidf.get_feature_names())

54777

In [12]:
import random

In [13]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names()[random_word_id])

canal
penetrate
divers
coiffures
dislike
providing
planner
entrants
twain
lateral


In [14]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names()[random_word_id])

swarm
chemical
opal
forearm
retire
stream
wagner
partridge
chanted
driverless


In [15]:
len(nmf_model.components_)

9

In [16]:
nmf_model.components_

array([[6.72681749e-04, 2.65840688e-01, 0.00000000e+00, ...,
        2.58461694e-03, 3.45272443e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.11487928e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.46683107e-03, 4.93217452e-02, 0.00000000e+00, ...,
        0.00000000e+00, 3.35930924e-06, 0.00000000e+00],
       [0.00000000e+00, 2.92470471e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.36036870e-03, 3.97037017e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [17]:
len(nmf_model.components_[0])

54777

In [18]:
single_topic = nmf_model.components_[0]

In [19]:
single_topic.argsort()

array([27388, 27082, 27079, ..., 19307, 36283, 42993])

In [22]:
# Word least representative of this topic
single_topic[27388]

0.0

In [24]:
# Word most representative of this topic
single_topic[42993]

2.31845168496178

In [28]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([54412, 26752, 47218, 33390, 36310, 28659, 53152, 19307, 36283,
       42993])

In [29]:
top_word_indices = single_topic.argsort()[-10:]

In [30]:
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

years
just
study
new
percent
like
water
food
people
says


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [31]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['year', 'university', 'workers', '000', 'company', 'years', 'just', 'study', 'new', 'percent', 'like', 'water', 'food', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['republicans', 'gop', 'pence', 'presidential', 'administration', 'election', 'obama', 'republican', 'white', 'donald', 'house', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['repeal', 'house', 'law', 'act', 'people', 'tax', 'plan', 'republicans', 'affordable', 'obamacare', 'coverage', 'medicaid', 'insurance', 'care', 'health']


THE TOP 15 WORDS FOR TOPIC #3
['iran', 'security', 'assad', 'aleppo', 'attack', 'korea', 'war', 'russian', 'forces', 'said', 'military', 'syrian', 'syria', 'russia', 'isis']


THE TOP 15 WORDS FOR TOPIC #4
['cruz', 'primary', 'election', 'democrats', 'percent', 'party', 'delegates', 'vote', 'state', 'democratic', 'hillary', 'campaign', 'voters', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['black', 'love', 'way', 'women', 't

### Attaching Discovered Topic Labels to Original Articles

In [32]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [33]:
dtm.shape

(11992, 54777)

In [34]:
len(npr)

11992

In [35]:
topic_results = nmf_model.transform(dtm)

In [36]:
topic_results.shape

(11992, 9)

In [37]:
topic_results[0]

array([0.        , 0.11571648, 0.00298438, 0.09629477, 0.0149703 ,
       0.        , 0.        , 0.        , 0.        ])

In [38]:
topic_results[0].round(2)

array([0.  , 0.12, 0.  , 0.1 , 0.01, 0.  , 0.  , 0.  , 0.  ])

In [39]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [40]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [41]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 7, 4, 0])

In [42]:
npr['Topic'] = topic_results.argmax(axis=1)

In [43]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",6
5,I did not want to join yoga class. I hated tho...,5
6,With a who has publicly supported the debunk...,0
7,"I was standing by the airport exit, debating w...",0
8,"If movies were trying to be more realistic, pe...",0
9,"Eighteen years ago, on New Year’s Eve, David F...",0
